In [19]:
# importing and unzipping our snake data (images in .zip)

import zipfile
import os

zip_path = 'snake_data.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./snake_data')

# folder structure
for root, dirs, files in os.walk("./snake_data", topdown=True):
    print(root)

./snake_data
./snake_data/snake_data
./snake_data/snake_data/snake_data
./snake_data/snake_data/snake_data/train
./snake_data/snake_data/snake_data/train/non_venemous
./snake_data/snake_data/snake_data/train/venemous
./snake_data/snake_data/snake_data/test
./snake_data/snake_data/snake_data/test/non_venemous
./snake_data/snake_data/snake_data/test/venemous
./snake_data/__MACOSX
./snake_data/__MACOSX/snake_data
./snake_data/__MACOSX/snake_data/snake_data
./snake_data/__MACOSX/snake_data/snake_data/train
./snake_data/__MACOSX/snake_data/snake_data/train/non_venemous
./snake_data/__MACOSX/snake_data/snake_data/test
./snake_data/__MACOSX/snake_data/snake_data/test/non_venemous
./snake_data/__MACOSX/snake_data/snake_data/test/venemous


In [20]:
import shutil
import os

# moving the "correct" nested folder up one level
nested_path = "./snake_data/snake_data/snake_data"
flat_path = "./snake_data_fixed"

shutil.move(nested_path, flat_path)

# makes directory nesting so image folders are in a clean and more accessible location for loading

'./snake_data_fixed'

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# defining our paths
train_dir = './snake_data_fixed/train'
test_dir = './snake_data_fixed/test'

# creating the data generators
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# above, we've loaded and labeled images from folders while scaling pixel values for model training

Found 2938 images belonging to 2 classes.
Found 1232 images belonging to 2 classes.


In [22]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# loads the MobileNetV2 model without the top layers (we’ll add our own output layers
base_model = MobileNetV2(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
base_model.trainable = False  # freezes the MobileNetV2 base model so pretrained weights don’t change during training

# builds our final model by new layers on top of the frozen base one
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')  # used forbinary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# summary: builds and compiles a CNN using MobileNetV2 base to classify venomous verrsus non-venomous snakes

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 128)               163968    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2422081 (9.24 MB)
Trainable params: 1640

In [23]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

# trains the model using the training data for 10 full passes, aka epochs, while checking accuracy on test data each time

Epoch 1/10
92/92 [==============================] - 10s 90ms/step - loss: 0.1994 - accuracy: 0.9217 - val_loss: 0.1037 - val_accuracy: 0.9610
Epoch 2/10
92/92 [==============================] - 7s 82ms/step - loss: 0.0768 - accuracy: 0.9731 - val_loss: 0.1224 - val_accuracy: 0.9545
Epoch 3/10
92/92 [==============================] - 7s 78ms/step - loss: 0.0532 - accuracy: 0.9850 - val_loss: 0.1019 - val_accuracy: 0.9667
Epoch 4/10
92/92 [==============================] - 7s 77ms/step - loss: 0.0372 - accuracy: 0.9888 - val_loss: 0.1140 - val_accuracy: 0.9635
Epoch 5/10
92/92 [==============================] - 7s 77ms/step - loss: 0.0279 - accuracy: 0.9915 - val_loss: 0.1307 - val_accuracy: 0.9594
Epoch 6/10
92/92 [==============================] - 7s 78ms/step - loss: 0.0236 - accuracy: 0.9915 - val_loss: 0.1208 - val_accuracy: 0.9651
Epoch 7/10
92/92 [==============================] - 7s 78ms/step - loss: 0.0164 - accuracy: 0.9939 - val_loss: 0.1361 - val_accuracy: 0.9635
Epoch 8/10
9

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np

# get true labels from the test generator
y_true = test_data.classes

# predict probabilities from the model
y_pred_probs = model.predict(test_data)

# convert probabilities to binary labels (as we use binary classification)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# specific evaluation metrics defined in our MI2
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = list(test_data.class_indices.keys())

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=labels))

# showing our confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
